# Overview
This notebook is designed to perform number sorting tasks using machine learning and computer vision techniques. It utilizes the Dorna robot, a camera, and object detection algorithms to sort objects based on their numbers.

## Imports and parameters

In [4]:
"""
import necessary modules
"""
from dorna2 import Dorna
from camera import Camera
from dorna_vision import Detection
import time

"""
parameters
"""
robot_ip_address = "192.168.254.87" # robot ip address
output_config = [0, 0, 1] # output in order (pin, off state, on state)
tcp = [0, 0, 44, 0, 0, 0] # tool length and position

place_positions = {
    "1": [156, 100, 10, 180, 0, 0],  # Position for placing chip 1
    "2": [156, 50, 10, 180, 0, 0],  # Position for placing chip 2
    "3": [156, 0, 10, 180, 0, 0],  # Position for placing chip 3
    "4": [156, -50, 10, 180, 0, 0],  # Position for placing chip 4
    "5": [156, -100, 10, 180, 0, 0],  # Position for placing chip 5
}
imaging_joint = [-12.392578, 45.219727, -132.319336, -13.31543, -6.196289, 0] # this is the place where we take a picture
speed = 0.5 # here set the speed of the robot
above = 20
sim=0

# detection parameters
detection_parameters = {'detection': {'cmd': 'od', 'path': 'Assets/game_chips.pkl', 'conf': 0.5, 'cls': []}, 'limit': {'area': [], 'aspect_ratio': [], 'xyz': [[180, 356], [-121, 121], [0, 20]], 'inv': 0}}


## Main loop

In [5]:
"""
initialize the robot, camera, and object detection
"""
robot = Dorna() # initialize robot
robot.connect(robot_ip_address) # connect to robot

camera = Camera() # initialize camera
camera.connect() # connect to camera

object_detection = Detection(camera=camera, robot=robot, **detection_parameters) # initialize the object detection

"""
set up the robot
"""
robot.set_output(output_config[0], output_config[1]) # set output
robot.set_motor(1) # turn on the robot motors
robot.sleep(1) # sleep to allow the robot to settle before moving
robot.jmove(imaging_joint)  #set safe initial position

"""
run the object detection and pick and place
"""
for i in range(10):  # range is set to 5 because we have 5 chips
    # always run this to make sure that the rbot is stationary before running the object detection
    time.sleep(0.1)

    # run the object detection
    result = object_detection.run()

    if result: # if there is an object detected

        pick_position = result[0]["xyz"] + [180, 0, 0] # get the pick position from the detection result
        object_class = result[0]["cls"] # get the class of the detected chip
        place_position = place_positions[object_class] # find the place position based on the chip's number

        # this executes the pick-and-place operation
        retval = robot.pick_n_place(
            pick_pose=pick_position, # pick position we got from detection
            place_pose=place_position, # place poistion which we got from the class
            end_joint=imaging_joint, # end position which we will call the imaging postion because after ending  we want to scan again
            tcp=tcp, # tcp is the tool length and postion which we deffined in the cfg cell
            speed=speed, # speed which we also defined in the cfg cell
            output_config = output_config, # output in order (pin, off state, on state)
            above=above,
            sim=sim,
            )

"""
it is necessary to close the robot, camera, and object detection
"""
robot.close() # close robot
camera.close() # close camera
object_detection.close() # close object detection